In [26]:
import pandas as pd
import numpy as np
import os
import torch
from torch import nn
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import ByteLevel
import random
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
import multiprocessing as mp
import threading
from PIL import Image
import cv2

In [2]:
TOTAL_PAYLOADS=100

with open("D:\\CODES\\Projects\\Ransomware_Desktop\\Tokens.json","r+") as Tokenizer:
    data=json.load(Tokenizer)

dic={0:"Aleta",
     1:"Bart",
     2:"BitPaymer",
     3:"crylock",
     4:"crypmic",
     5:"cryptFile2",
     6:"CTBlocker",
     7:"DMAlocker",
     8:"eris",
     9:"GlobeImposter",
     10:"hive",
     11:"maze",
     12:"maktub",
     13:"mole",
     14:"MRCR",
     15:"netwalker",
     16:"phobos",
     17:"ransomX",
     18:"razi",
     19:"revenge",
     21:"scarab",
     22:"teslaCrypt",
     23:"zeus"}

In [3]:
dir_path=r"D:\DATASETS\ransomware"
Ransomware_csv=r"D:\DATASETS\ransomware\PCAP_data"
data_dir=os.path.join(dir_path,"CNN_dataset\Tokenized")
ransomware_SMB=os.path.join(data_dir,"ransomware_SMB")
Good_SMB=os.path.join(data_dir,"Good_SMB")

In [4]:
#Ransomware Family Paths
Aleta_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Aleta\Aleta_with_payload.csv"
Bart_csv_path=r"D:\DATASETS\ransomware\PCAP_data\bart\bart_with_payload.csv"
BitPaymer_csv_path=r"D:\DATASETS\ransomware\PCAP_data\bitPaymer\bitPaymer_with_payload.csv"
crylock_csv_path=r"D:\DATASETS\ransomware\PCAP_data\CryLock\CryLock_with_payload.csv"
crypmic_csv_path=r"D:\DATASETS\ransomware\PCAP_data\CrypMIC\CrypMIC_with_payload.csv"
cryptFile2_csv_path=r"D:\DATASETS\ransomware\PCAP_data\CryptFile2\CryptFile2_with_payload.csv"
CTBlocker_csv_path=r"D:\DATASETS\ransomware\PCAP_data\CTBLocker\CTBLocker_with_payload.csv"
DMAlocker_csv_path=r"D:\DATASETS\ransomware\PCAP_data\DMALocker\DMALocker_with_payload.csv"
eris_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Eris\Eris_with_payload.csv"
GlobeImposter_csv_path=r"D:\DATASETS\ransomware\PCAP_data\GlobeImposter_06052017.pcap\GlobeImposter_06052017.pcap_with_payload.csv"
hive_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Hive\Hive_with_payload.csv"
maze_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Maze\Maze_with_payload.csv"
maktub_csv_path=r"D:\DATASETS\ransomware\PCAP_data\maktub\maktub_with_payload.csv"
mole_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Mole\Mole_with_payload.csv"
MRCR_csv_path=r"D:\DATASETS\ransomware\PCAP_data\MRCR\MRCR_with_payload.csv"
netwalker_csv_path=r"D:\DATASETS\ransomware\PCAP_data\netwalker_21012021.pcap\netwalker_21012021.pcap_with_payload.csv"
phobos_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Phobos\Phobos_with_payload.csv"
ransomX_csv_path=r"D:\DATASETS\ransomware\PCAP_data\RansomX\RansomX_with_payload.csv"
razi_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Razi\Razi_with_payload.csv"
revenge_csv_path=r"D:\DATASETS\ransomware\PCAP_data\revenge\revenge_with_payload.csv"
scarab_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Scarab\Scarab_with_payload.csv"
teslaCrypt_csv_path=r"D:\DATASETS\ransomware\PCAP_data\TeslaCrypt\TeslaCrypt_with_payload.csv"
zeus_csv_path=r"D:\DATASETS\ransomware\PCAP_data\Zeus\Zeus_with_payload.csv"

In [5]:
Aleta=pd.read_csv(Aleta_csv_path)
Bart=pd.read_csv(Bart_csv_path)
BitPaymer=pd.read_csv(BitPaymer_csv_path)
crylock=pd.read_csv(crylock_csv_path)
crypmic=pd.read_csv(crypmic_csv_path)
cryptFile2=pd.read_csv(cryptFile2_csv_path)
CTBlocker=pd.read_csv(CTBlocker_csv_path)
DMAlocker=pd.read_csv(DMAlocker_csv_path)
eris=pd.read_csv(eris_csv_path)
GlobeImposter=pd.read_csv(GlobeImposter_csv_path)
hive=pd.read_csv(hive_csv_path)
maze=pd.read_csv(maze_csv_path)
maktub=pd.read_csv(maktub_csv_path)
mole=pd.read_csv(mole_csv_path)
MRCR=pd.read_csv(MRCR_csv_path)
netwalker=pd.read_csv(netwalker_csv_path)
phobos=pd.read_csv(phobos_csv_path)
ransomX=pd.read_csv(ransomX_csv_path)
razi=pd.read_csv(razi_csv_path)
revenge=pd.read_csv(revenge_csv_path)
scarab=pd.read_csv(scarab_csv_path)
teslaCrypt=pd.read_csv(teslaCrypt_csv_path)
zeus=pd.read_csv(zeus_csv_path)

In [6]:
'''
1.extract payloads
2.drop NaN
3.cropping and padding
4.Tokenize
5.to image
6.save image in a proper folder

Aleta=Aleta.dropna(subset=["payload"])
Bart=Bart.dropna(subset=["payload"])
BitPaymer=BitPaymer.dropna(subset=["payload"])
crylock=crylock.dropna(subset=["payload"])
crypmic=crypmic.dropna(subset=["payload"])
cryptFile2=cryptFile2.dropna(subset=["payload"])
CTBlocker=CTBlocker.dropna(subset=["payload"])
DMAlocker=DMAlocker.dropna(subset=["payload"])
eris=eris.dropna(subset=["payload"])
GlobeImposter=GlobeImposter.dropna(subset=["payload"])
hive=hive.dropna(subset=["payload"])
maze=maze.dropna(subset=["payload"])
maktub=maktub.dropna(subset=["payload"])
mole=mole.dropna(subset=["payload"])
MRCR=MRCR.dropna(subset=["payload"])
netwalker=netwalker.dropna(subset=["payload"])
phobos=phobos.dropna(subset=["payload"])
ransomX=ransomX.dropna(subset=["payload"])
razi=razi.dropna(subset=["payload"])
revenge=revenge.dropna(subset=["payload"])
scarab=scarab.dropna(subset=["payload"])
teslaCrypt=teslaCrypt.dropna(subset=["payload"])
zeus=zeus.dropna(subset=["payload"])'''

'\n1.extract payloads\n2.drop NaN\n3.cropping and padding\n4.Tokenize\n5.to image\n6.save image in a proper folder\n\nAleta=Aleta.dropna(subset=["payload"])\nBart=Bart.dropna(subset=["payload"])\nBitPaymer=BitPaymer.dropna(subset=["payload"])\ncrylock=crylock.dropna(subset=["payload"])\ncrypmic=crypmic.dropna(subset=["payload"])\ncryptFile2=cryptFile2.dropna(subset=["payload"])\nCTBlocker=CTBlocker.dropna(subset=["payload"])\nDMAlocker=DMAlocker.dropna(subset=["payload"])\neris=eris.dropna(subset=["payload"])\nGlobeImposter=GlobeImposter.dropna(subset=["payload"])\nhive=hive.dropna(subset=["payload"])\nmaze=maze.dropna(subset=["payload"])\nmaktub=maktub.dropna(subset=["payload"])\nmole=mole.dropna(subset=["payload"])\nMRCR=MRCR.dropna(subset=["payload"])\nnetwalker=netwalker.dropna(subset=["payload"])\nphobos=phobos.dropna(subset=["payload"])\nransomX=ransomX.dropna(subset=["payload"])\nrazi=razi.dropna(subset=["payload"])\nrevenge=revenge.dropna(subset=["payload"])\nscarab=scarab.drop

In [7]:
Aleta=Aleta[Aleta["protocol"]==6]
Bart=Bart[Bart["protocol"]==6]
BitPaymer=BitPaymer[BitPaymer["protocol"]==6]
crylock=crylock[crylock["protocol"]==6]
crypmic=crypmic[crypmic["protocol"]==6]
cryptFile2=cryptFile2[cryptFile2["protocol"]==6]
CTBlocker=CTBlocker[CTBlocker["protocol"]==6]
DMAlocker=DMAlocker[DMAlocker["protocol"]==6]
eris=eris[eris["protocol"]==6]
GlobeImposter=GlobeImposter[GlobeImposter["protocol"]==6]
hive=hive[hive["protocol"]==6]
maze=maze[maze["protocol"]==6]
maktub=maktub[maktub["protocol"]==6]
mole=mole[mole["protocol"]==6]
MRCR=MRCR[MRCR["protocol"]==6]
netwalker=netwalker[netwalker["protocol"]==6]
phobos=phobos[phobos["protocol"]==6]
ransomX=ransomX[ransomX["protocol"]==6]
razi=razi[razi["protocol"]==6]
revenge=revenge[revenge["protocol"]==6]
scarab=scarab[scarab["protocol"]==6]
teslaCrypt=teslaCrypt[teslaCrypt["protocol"]==6]
zeus=zeus[zeus["protocol"]==6]

In [8]:
Aleta=Aleta["payload"]
Bart=Bart["payload"]
BitPaymer=BitPaymer["payload"]
crylock=crylock["payload"]
crypmic=crypmic["payload"]
cryptFile2=cryptFile2["payload"]
CTBlocker=CTBlocker["payload"]
DMAlocker=DMAlocker["payload"]
eris=eris["payload"]
GlobeImposter=GlobeImposter["payload"]
hive=hive["payload"]
maze=maze["payload"]
maktub=maktub["payload"]
mole=mole["payload"]
MRCR=MRCR["payload"]
netwalker=netwalker["payload"]
phobos=phobos["payload"]
ransomX=ransomX["payload"]
razi=razi["payload"]
revenge=revenge["payload"]
scarab=scarab["payload"]
teslaCrypt=teslaCrypt["payload"]
zeus=zeus["payload"]

In [9]:
l=[
    Aleta,
Bart,
BitPaymer,
crylock,
crypmic,
cryptFile2,
CTBlocker,
DMAlocker,
eris,
GlobeImposter,
hive,
maze,
maktub,
mole,
MRCR,
netwalker,
phobos,
ransomX,
razi,
revenge,
scarab,
teslaCrypt,
zeus]
lengths=[]
for i in range(len(l)):
    fam=l[i]
    lengths.append(len(fam))

In [10]:
'''PreProcessing functions to crop and padd 784 and 1568 hexa codes'''

def preprocessPayload(l):
    croppedlist=l[:784]
    if(len(croppedlist)<784):
        zeroes=784-len(croppedlist)
        for i in range(zeroes):
            croppedlist.append(0)
    return croppedlist
    
def pairer(string):
    pairs=[]
    for i in range(0,len(string)-1,1):
        pair=string[i]+string[i+1]
        pairs.append(pair)
    return pairs
    
def Tokenize(string):
    l=[]
    c=0
    pairs=pairer(string)
    for i in pairs:
        l.append(data[i])
    return l               

def ToimageArray(string):
    x=Tokenize(string)
    y=preprocessPayload(x)
    return np.reshape(y,(28,28))
        
def chunker(family_list,TOTAL_PAYLOADS):
    chunks=[]
    for i in range(0,len(family_list),TOTAL_PAYLOADS):
        chunk=[]
        for j in range(i,TOTAL_PAYLOADS,1):
            if(j>=len(family_list)):
                break
            chunk.append(family_list[j])
        chunks.append(chunk)
    return chunks

In [27]:
def lToImgArray(l,start,end,queue):
    FINAL=[]
    for i in range(start,end):
        fam=[]
        for j in range(0,len(l[i])):
            x=ToimageArray(l[i].iloc[j])
            fam.append(x)
        ch=chunker(fam,TOTAL_PAYLOADS)
        FINAL.append(ch)
    queue.put(FINAL) 


FINAL=[]
for i in tqdm(range(len(l)), desc="Converting Families to Image Arrays"):
    c=1
    fam=[]
    la=list(l[i])
    #ch=chunker(la,TOTAL_PAYLOADS)
    for j in range(len(la)):
        payload=la[j]
        if(type(payload)==float):
            payload=str(payload)
        a=ToimageArray(payload)
        path=ransomware_SMB + f"{dic[i]}"+ f"_{c}.png"
        cv2.imwrite(path, test)
        c+=1

Converting Families to Image Arrays:  30%|████████████▏                           | 7/23 [3:10:29<7:15:25, 1632.84s/it]

KeyboardInterrupt



In [ ]:
'''if __name__=="__main__":
    queue=mp.Queue()
    processes = [
        mp.Process(target=lToImgArray, args=(l, 0, 3, queue)),
        mp.Process(target=lToImgArray, args=(l, 3, 6, queue)),
        mp.Process(target=lToImgArray, args=(l, 6, 9, queue)),
        mp.Process(target=lToImgArray, args=(l, 9, 12, queue)),
        mp.Process(target=lToImgArray, args=(l, 12, 15, queue)),
        mp.Process(target=lToImgArray, args=(l, 15, 18, queue)),
        mp.Process(target=lToImgArray, args=(l, 18, 21, queue)),
        mp.Process(target=lToImgArray, args=(l, 21, 24, queue)),
    ]
    for p in processes:
        p.start()
        
    results = []
    for _ in processes:
        results.append(queue.get())  # Retrieve results from the queue

    # Wait for all processes to complete
    for p in processes:
        p.join()

    # Combine results
    print("Results:", results)'''

In [ ]:


'''for i in tqdm(range(len(FINAL)), desc="Processing Families"):
    c=0
    for j in range(FINAL[i]):
        imgarray=FINAL[i][j]
        image = Image.fromarray(array)
        path=ransomware_SMB + f"{dic[i]}"+ f"_{c}"
        image.save(path)
        c+='''

Aleta=
Bart=
BitPaymer=
crylock=
crypmic=
cryptFile2=
CTBlocker=
DMAlocker=
eris=
GlobeImposter=
hive=
maze=
maktub=
mole=
MRCR=
netwalker=
phobos=
ransomX=
razi=
revenge=
scarab=
teslaCrypt=
zeus=

In [28]:
len(l[7])

1462287